In [16]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
x_train = np.asarray([])
y_train = np.asarray([])
x_test = np.asarray([])
y_test = np.asarray([])

In [17]:
class Data(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.x.shape[0]
    
    def __get_item__(self, indx):
        return self.x[indx], self.y[indx]
    
    def __len__(self):
        return self.len

In [18]:
BATCH_SIZE = 256

In [ ]:
train_data = Data(x_train, y_train)
train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)

test_data = Data(x_test, y_test)
test_loader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = True)

In [8]:
class Likeability(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Likeability, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.layers.append(nn.Linear(input_dim, hidden_dim))
            elif i == num_layers-1:
                self.layers.append(nn.Linear(hidden_dim, 1))
            else:
                self.layers.append(nn.Linear(hidden_dim, hidden_dim))
            nn.init.kaiming_uniform_(self.layers[-1].weight, nonlinearity="relu")
    
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            x = x.relu()
            if i != len(self.layers) - 1:
                x = F.dropout(x, p=0.5, training=self.training)
        return x

In [9]:
model = Likeability(25, 64, 3)

In [14]:
model.to(device)

Likeability(
  (layers): ModuleList(
    (0): Linear(in_features=25, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [15]:
EPOCHS = 100
LR = 2e-3
LOSS = nn.MSELoss()
OPT = optim.Adam(model.parameters(), lr = LR)

In [ ]:
losses = []

for epoch in range(EPOCHS):
    for x, y in train_loader:
        OPT.zero_grad()
        pred = model(x)
        loss = LOSS(pred, y.unsqueeze(-1))
        losses.append(loss)
        loss.backward()
        OPT.step()

In [ ]:
y_pred = []
y_test = []
preds = []

with torch.no_grad():
    for x, y in test_loader:
        pred = model(x)
        y_pred.append(pred)
        y_test.append(y)
        preds.append(int(pred == y))

In [ ]:
sum(pred)/len(preds)